In [1]:
%matplotlib inline
from pprint import pprint
import itertools

from market import Returns_US_1871
from simulate import simulate_withdrawals
from harvesting import N_60_RebalanceHarvesting
from metrics import average, mean, median
from withdrawal import ConstantWithdrawals
import pandas

from adt import AnnualChange
import numpy

In [2]:
class Monthly_Returns_US_1871:
    def __init__(self, wrap=False):
        self.dataframe = pandas.read_csv('schiller_monthly.csv')
        self.years_of_data = len(self.dataframe)

    def __iter__(self):
        return self.iter_from(1871)

    def iter_from(self, year, length=None):
        start = (year * 12) - (1871 * 12)
        count = 0
        for row in self.dataframe.iloc[start:].iterrows():
            yield row[1]
            count += 1
            if length != None and count >= length:
                raise StopIteration


In [3]:
m = Monthly_Returns_US_1871()
count = 1
trigger = []

import datetime

def diff(x, y):
    (x_year, x_month) = map(lambda n: int(n), x.split('.'))
    (y_year, y_month) = map(lambda n: int(n), y.split('.'))

    dx = datetime.date(x_year, x_month, 1)
    dy = datetime.date(y_year, y_month, 1)
    return dy - dx

after_days = []
gain = []

for (_, i) in m.dataframe.iloc[0:].iterrows():
    date = str(i['Date'])
    index = float(i['P'])

    if count == 1:
        start_of_year = index
        
        for (d, i) in trigger:
            if index >= i * 1.20:
                after_days.append(diff(d, date).days)
                gain.append((index - i) / i)
                #print('Harvest %s in %s' % (d, date))
                trigger.remove((d,i))

        trigger.append((date, index))

    count += 1
    if count == 13:
        count = 1

print(numpy.std(after_days))
print(numpy.mean(after_days))
print(numpy.median(after_days))
print(numpy.median(gain))

2223.02446985
2484.64084507
1461.0
0.3494318973


In [4]:
after_days = []
count = 1
gain = []

for (_, i) in m.dataframe.iloc[0:].iterrows():
    date = str(i['Date'])
    index = float(i['P'])
        
    for (d, i) in trigger:
        if index >= i * 1.20:
            after_days.append(diff(d, date).days)
            gain.append((index - i) / i)
            #print('Harvest %s in %s' % (d, date))
            trigger.remove((d,i))

    if count == 1:
        trigger.append((date, index))

    count += 1
    if count == 13:
        count = 1

print(numpy.std(after_days))
print(numpy.mean(after_days))
print(numpy.median(after_days))
print(numpy.median(gain))

1911.21296422
1693.13888889
761.5
0.2184182531
